In [1]:
import os
import numpy as np
from openbabel import openbabel as ob
from optimization_methods import write_xyz
from cclib.io import ccread
os.chdir('Resources/tri')
data = ccread('TSCoDe_checkpoint_May_27_13-49.xyz')
constraints = [[6, 66], [56, 71], [23, 34]]

In [12]:
def OB_MM_POPT(structures, atomnos, constrained_indexes, method='UFF'):
        '''
        return : MM-optimized structure (UFF/MMFF)
        '''

        filename='temp_ob_in.xyz'

        with open(filename, 'w') as f:
            for structure in structures:
                write_xyz(structure, atomnos, f)

        outname = 'temp_ob_out.xyz'

        # Standard openbabel molecule load
        conv = ob.OBConversion()
        conv.SetInAndOutFormats('xyz','xyz')
        mol = ob.OBMol()
        more = conv.ReadFile(mol, filename)
        i = 0

        # Define constraints
        constraints = ob.OBFFConstraints()

        for a, b in constrained_indexes:

            first_atom = mol.GetAtom(int(a+1))
            length = first_atom.GetDistance(int(b+1))

            constraints.AddDistanceConstraint(int(a+1), int(b+1), length)       # Angstroms
            # constraints.AddAngleConstraint(1, 2, 3, 120.0)      # Degrees
            # constraints.AddTorsionConstraint(1, 2, 3, 4, 180.0) # Degrees

        # Setup the force field with the constraints
        forcefield = ob.OBForceField.FindForceField(method)
        forcefield.Setup(mol, constraints)
        forcefield.SetConstraints(constraints)

        # Do a 500 steps conjugate gradient minimization
        # (or less if converges) and save the coordinates to mol.
        forcefield.ConjugateGradients(500)
        forcefield.GetCoordinates(mol)

        # Write the mol to a file
        conv.WriteFile(mol,outname)
        # conv.CloseOutFile()

        return ccread(outname).atomcoords

In [13]:
%%time
newcoords = OB_MM_POPT(data.atomcoords, data.atomnos, constraints)

PermissionError: [WinError 32] Impossibile accedere al file. Il file è utilizzato da un altro processo: 'ob_temp_in.xyz'